In [1]:
from scraper import *

# 机构

In [ ]:
df_companies = Company().get_company_list()

# 人员

In [24]:
def get_people_list(company_ID):
    # 启动浏览器
    # TODO：添加Headless选项
    driver = webdriver.Chrome()
    driver.get('http://person.sac.net.cn/pages/registration/sac-publicity-finish.html?aoiId='+company_ID)

    # 确认页数
    def get_pages_num(driver):
        pages_num = 0 # 保证即便确认页数错误，仍然可以获得第一页而不会终止程序
        try:
            pages_num = int(float(driver.find_element_by_id('sp_1').text.strip()))
        except Exception as e:
            print ('请修改确认页数的ID标签，错误代码为：', e)
        finally:
            return pages_num
        
    # 翻页
    def change_page(driver):
        driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='无'])[100]/following::span[6]").click()
        time.sleep(2) # 防止运行过快被封

    # 爬取单页
    def parser(driver, data):
        content = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(content, 'lxml')
        table = soup.find('div', class_='listable_out').find('table',class_='ui-jqgrid-htable')

        # 得到表格标题
        list_columns = []
        for column in table.find_all('tr')[0].find_all('th'):
            list_columns.append(column.get_text().strip())

        # 得到表格数据
        for row in soup.find('div', class_='ui-jqgrid-bdiv').find_all('tr', id=True):
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])  # Get rid of empty values

        return data, list_columns

    # 主程序部分
    pages_num = get_pages_num(driver)

    data = []
    data, list_columns = parser(driver, data)
    for i in tqdm(range(pages_num-1)): ## 最后一页不用点，可以换成while循环更优雅
        change_page(driver)
        data, list_columns = parser(driver, data)

    driver.quit()
    df_companies = pd.DataFrame(data, columns=list_columns)

    # 描述表格数据
    print('表格共有{}列，{}行。'.format(len(df_companies.columns), len(df_companies)))
    display(df_companies)
    return df_companies

In [25]:
df_companies = get_people_list(company_ID='1999139')

请修改确认页数的ID标签，错误代码为： could not convert string to float: 
表格共有11列，0行。


0it [00:00, ?it/s]


,PPP_ID,姓名,性别,学历,证书编号,执业机构,执业岗位,证书取得日期,证书有效截止日期,注册变更记录,诚信记录


In [28]:
# 调试代码

driver = webdriver.Chrome()
driver.get('http://person.sac.net.cn/pages/registration/sac-publicity-finish.html?aoiId='+'1999139')

# 确认页数
def get_pages_num(driver):
    pages_num = 0 # 保证即便确认页数错误，仍然可以获得第一页而不会终止程序
    try:
        print (driver.find_element_by_id('sp_1').text.strip())
        pages_num = int(float(driver.find_element_by_id('sp_1').text.strip()))
    except Exception as e:
        print ('请修改确认页数的ID标签，错误代码为：', e)
    finally:
        return pages_num

# 翻页
def change_page(driver):
    driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='无'])[100]/following::span[6]").click()
    time.sleep(2) # 防止运行过快被封

# 爬取单页
def parser(driver, data):
    content = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    table = soup.find('div', class_='listable_out').find('table',class_='ui-jqgrid-htable')

    # 得到表格标题
    list_columns = []
    for column in table.find_all('tr')[0].find_all('th'):
        list_columns.append(column.get_text().strip())

    # 得到表格数据
    for row in soup.find('div', class_='ui-jqgrid-bdiv').find_all('tr', id=True):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])  # Get rid of empty values

    return data, list_columns

# 主程序部分
pages_num = get_pages_num(driver)


请修改确认页数的ID标签，错误代码为： could not convert string to float: 


In [30]:
float(driver.find_element_by_id('sp_1').text.strip())

8.0